In [7]:
# !pip install langchain google-generativeai pydantic langchain-community langchain-google-genai langchain-core

In [8]:
import os
print(f"Current working directory is: {os.getcwd()}")

Current working directory is: c:\Users\mishr\SurajMishra\MyWork\code_review_agent


#### Code Review Prompts

In [9]:
SYSTEM_MESSAGE = """You are an expert in Power Automate and automation workflows. 
Review the provided Power Automate code and assess it for correctness, efficiency, and security. 
The code can be present in the form of xml or json files which are generated when we export the workflow from Power Automate.

Provide your output in the following JSON structure:
{{
  "review_status": boolean,  # True if code is good, False if issues are found
  "problems_identified": [   # List of problems in the code
    "Describe the problem briefly"
  ],
  "recommendations": [       # List of recommendations to improve code
    "Provide recommendations for improvement"
  ]
}}
"""

INSTRUCTIONS = """# **Power Automate Best Practices for Code Review:**

## **1️⃣ Naming Conventions**
- Use **clear, meaningful names** (e.g., `SendApprovalEmail`, not `action1`).  
- Follow **PascalCase** for flow names (`NewCustomerFlow`), and **camelCase** for variables (`var_userEmail`).  

## **2️⃣ Structure & Organization**
- **Group related actions** using **Scope** (e.g., `Scope: File Operations`).  
- Keep flows **modular** and use **child flows** for reusability.  

## **3️⃣ Error Handling**
- Implement **“Run After”** settings for error handling.  
- Use **Try-Catch Scopes** to log errors and send alerts.  
- Store error logs in **SharePoint, Dataverse, or Log Analytics**.  

## **4️⃣ Performance Optimization**
- **Filter data before loops** (use OData queries instead of iterating all items).  
- **Use parallel branches** for independent tasks.  
- **Minimize API calls** by selecting only required fields.  

## **5️⃣ Security & Access Control**
- Use **service accounts**, not personal accounts, for connections.  
- Restrict permissions to **least privilege access**.  
- Store sensitive data in **Azure Key Vault or environment variables**.  

## **6️⃣ Version Control & Documentation**
- **Enable version history** and add meaningful commit messages.  
- Add **comments in actions** explaining logic.  
- Maintain detailed documentation in **SharePoint, OneNote, or Confluence**.  

## **7️⃣ Code Efficiency**
- Use **expressions** instead of multiple actions (`'Hello ' & var_userName`).  
- Avoid redundant steps; consolidate where possible.  

This concise guide ensures **scalable, secure, and efficient** Power Automate workflows. 🚀

"""

HUMAN_MESSAGE = """Code to review:\n\n{code}"""


#### Input Output Schemas

In [10]:
from pydantic import BaseModel
from typing import List, Union


class CodeReviewInput(BaseModel):
    code: str  # Raw Power Automate code as string


class CodeReviewOutput(BaseModel):
    review_status: bool  # True if code is acceptable, False otherwise
    problems_identified: List[str]  # List of issues found
    recommendations: List[str]  # Suggestions for improvement


In [11]:
import os
import json

from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Load the environment variables
load_dotenv('.env')

# Define the Google Generative AI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=2000,
    timeout=45,
    max_retries=2,

)

def run_review_chain(code):

    # Define Output Parser
    parser = PydanticOutputParser(pydantic_object=CodeReviewOutput)

    # Define Prompt Template
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                SYSTEM_MESSAGE,
            ),
            ("human", INSTRUCTIONS),
            ("human", HUMAN_MESSAGE),
        ]
    )

    # Build LangChain Pipeline
    chain = prompt | llm | parser
    output = chain.invoke({"code": code})
    return output.dict()


In [ ]:

# Input & Output Directories
input_folder = "inputs"
output_folder = "outputs"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Process each file in inputs folder
for file in os.listdir(input_folder):
    if file.endswith(".json"):  # Assuming Power Automate code is JSON formatted
        file_path = os.path.join(input_folder, file)

        with open(file_path, "r", encoding="utf-8") as f:
            code_content = f.read()
    
    elif file.endswith(".xml"):  # Assuming Power Automate code is JSON formatted
        file_path = os.path.join(input_folder, file)

        with open(file_path, "r", encoding="utf-8") as f:
            code_content = f.read()

        # Run Code Review
        review_result = run_review_chain(code=code_content)

        print(f"File {file_path} review done!!\n\n{review_result}")

        # Save Output
        op_file_name = file.split(".")[0]
        output_path = os.path.join(output_folder, f"review_{op_file_name}.json")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(review_result, f, indent=4)

print("Code review completed. Check the outputs folder.")
